# **Tarea 4 - CC6205 Natural Language Processing 📚**

**Integrantes: Gerard Cathalifaud Salazar, Juan Pablo Herrera Pizarro**

**Fecha límite de entrega 📆:** Martes 13 de junio.

**Tiempo estimado de dedicación:**

Bienvenid@s a la cuarta tarea del curso de Natural Language Processing (NLP).
En esta tarea estaremos tratando el problema de **tagging** (generación de secuencias de etiquetas del mismo largo que la secuencia de input), el uso de **Convolutional Neural Networks** y **Recurrent Neural Networks**, e implementaremos una red usando PyTorch.

Usen $\LaTeX$ para las fórmulas matemáticas. En la parte de programación pueden usar lo que quieran, pero la [Auxiliar 3](https://youtu.be/36WTXvg3zh0) les puede ser de *gran ayuda*.

**Instrucciones:**
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación.
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso.

Si aún no han visto las clases, se recomienda visitar los links de las referencias.

**Referencias:**

- [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/Tjgb-yQOg54), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)
- [MEMMs and CRFs](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-CRF.pdf): [notes 1](http://www.cs.columbia.edu/~mcollins/crf.pdf), [notes 2](http://www.cs.columbia.edu/~mcollins/fb.pdf), [video 1](https://youtu.be/qlI-4lSUDkg), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/ZpUwDy6o28Y)
- [Convolutional Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-CNN.pdf): [video](https://youtu.be/lLZW5Fn40r8)
- [Recurrent Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-RNN.pdf): [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)

# Hidden Markov Models (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Field(CRF) (1,5 puntos)

### Pregunta 1 (1 pt)
Para un problema de POS tagging se define el conjunto de etiquetas $S = \{ \text{DET}, \text{NOUN}, \text{VERB}, \text{ADP} \}$ y se tiene un Hidden Markov Model con los siguientes parámetros estimados a partir de un corpus de entrenamiento:

\begin{equation}
\begin{split}
q(\text{NOUN}| \text{ VERB}, \text{DET}) &= 0.3 \\
q(\text{NOUN}|\ w, \text{DET}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
q(\text{DET}| \text{ VERB}, \text{NOUN}) &= 0.4 \\
q(\text{DET}|\ w, \text{NOUN}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
e(the|\text{ DET}) &= 0.5 \\
e(pasta|\text{ NOUN}) &= 0.6
\end{split}
\end{equation}

Luego para la oración: `the man is pouring sauce on the pasta`, se tiene una tabla de programación dinámica con los siguientes valores:
\begin{equation}
\begin{split}
\pi(7,\text{DET},\text{DET})&=0.1\\
\pi(7,\text{NOUN},\text{DET})&=0.2\\
\pi(7,\text{VERB},\text{DET})&=0.01\\
\pi(7,\text{ADP},\text{DET})&=0.5
\end{split}
\end{equation}

Con esta información, calcule el valor de $\pi(8,\text{DET},\text{NOUN})$. Puede dejar el resultado expresado como una fracción.


**Respuesta**

Se desea determinar:

$\pi (8,\text{DET},\text{NOUN})$

Según el algoritmo de Viterbi, que se aprovecha de una definición recursiva de $\pi$, se tiene que:

$\pi (k,u,v) = \max_{w\in S_{k-2}}(\pi(k-1,w,u)\times q(v|w,u)\times e(x_k|v))$

Entonces el cálculo se resumen en:

$\pi (8,\text{DET},\text{NOUN}) = \max_{w\in S}(\pi(7,w,\text{DET})\times q(\text{NOUN}|w,\text{DET})\times e(\text{pasta}|\text{NOUN}))$

Notar que la expresión $q(\text{NOUN}|w,\text{DET})=0,\forall w\in S, w \neq \text{VERB}$ y $q(\text{NOUN}|\text{VERB},\text{DET})=0.3$, por lo que todas las expresiones con $w$ distinto de $\text{VERB}$ serán 0 y el máximo se obtendrá con $w=\text{VERB}$

$\pi (8,\text{DET},\text{NOUN}) = \pi(7,\text{VERB},\text{DET})\times q(\text{NOUN}|\text{VERB},\text{DET})\times e(\text{pasta}|\text{NOUN})$

$\pi (8,\text{DET},\text{NOUN}) = 0.01 \times 0.3 \times 0.6$

$\pi (8,\text{DET},\text{NOUN}) = 0.0018$

### Pregunta 2 (0.5 pts)
Comente  sobre las similitudes o diferencias entre los HMMs, MEMMs y CRFs. Para esto, responda las siguientes preguntas.

#### 2.1. ¿Para qué tipo de tarea sirven? Dé dos ejemplo de este tipo de tarea y descríbalos brevemente. (0.1 pts)

**Respuesta:**

Estos modelos buscan resolver las tareas de taggings, es decir, dada una secuencia de entrada (en PLN es un documento) entregar una etiqueta para cada una de estas entradas (palabras), como ejemplo se tienen los clásicos problemas de:

*   part-of-speech tagging (POS): Que consiste en categorizar gramaticalmente cada palabra en su función del lenguaje.
*   Named Entity Recognition (NER): Reconocer que palabras se refieren a alguna entidad (y a qué tipo) y cuales no.



#### 2.2. ¿Qué modelos usan features? ¿Qué ventajas conlleva esto? (0.1 pts)

**Respuesta:**

Los modelos MEMMs y CRFs presentan vectores de features. Este tiene la ventaja de realizar una representación más completa que el modelo HMMs que no lo utiliza, pues le agrega sensibilidad a partes específicas del texto, como la presencia de bigramas, la existencia de prefijos o sufijos en la palabra, o la presencia de palabras en todo el texto, reflejando mayor información para resolver problemas en PLN.

#### 2.3. ¿Cómo maneja cada uno de los modelos las palabras con baja frecuencia en el set de train? (0.1 pts)

**Respuesta:**

Si existe baja frecuencia de palabras en el set de entrenamiento, los modelos actúan de la siguiente forma:

1.   HMMs:
Divide el vocabulario en palabras frecuentes (>5 por ej) y palabras poco frecuentes (el resto), y a este último lo mapea a un conjunto finito pequeño que los agrupe para así aumentar su frecuencia y procesarlos.

2.   MEMMs y CRFs:
Como utilizan un cálculo similar al softmax multiclase, se puede utilizar también estrategias como suavizado de Laplace, que consiste en sumar 1 a todas las frecuencias, garantizando que toda probabilidad sea distinta de 0 y que no haya indeterminaciones. También se pueden aplicar features que otorguen información adicional a las palabras poco frecuentes, como la presencia de prefijos/sufijos.

#### 2.4. ¿Qué le permite a los CRF realizar decisiones globales? ¿Qué diferencia con respecto a los MEMMs permite lograr esto? ¿Por qué los HMMs tampoco son capaces de tomar decisiones globales? (0.1 pts)

**Respuesta:**

CRF utiliza un vector de features que toma toda la sencuencia de entrada junto a toda una secuencia de etiquetas y normaliza por el espacio de todas las secuencias posibles, generando así una cobertura global para las decisiones. Por otro lado, MEMMs los va tomando localmente por cada paso de secuencia, en la posición i-ésima. Los HMMs utilizan un supuesto de Markov, solo teniendo en cuenta las últimas 2 etiquetas anteriores, por lo que también solo considera información a nivel local.

#### 2.5 Dado una secuencia de $x_1, ..., x_m$ ¿Cuántas posibles secuencias de etiquetas se pueden generar para un conjunto de etiquetas $S$ con $|S|=k$ ? ¿Analizarlas todas sería computacionalmente tratable? (0.1 pts)

**Respuesta:**

Para una secuencia de $m$ palabras, el conjunto de posibles secuencias de etiquetas es $S^m$ y tiene tamaño $k^m$. Esto es de orden exponencial respecto a $m$ por lo que es computacionalmente intratable analizar todas las posibles secuencias. Sin embargo, mediante supuestos apropiados y algoritmos que los aprovechen como Viterbi, es posible tener entrenamientos y decoding eficientes con este conjunto.

# Convolutional Neural Networks (0,5 puntos)

### Pregunta 3 (0,5 puntos)

Considere la frase $w_{1..7}=$ `El agua moja y el fuego quema` $=[El, agua, moja, y, el, fuego, quema]$.

La siguiente matriz de embeddings, donde la i-ésima fila corresponde al vector de embedding de la i-ésima palabra, ordenadas según aparecen en la frase. (vectores de largo 2).
\begin{equation}
E = \begin{pmatrix}
2 & 2\\
0 & -2\\
0 & 1\\
-2 & 1\\
1 & 0\\
-1 & 1\\
1 & 1
\end{pmatrix}
\end{equation}

Los siguientes 3 filtros
\begin{equation}
U = \begin{pmatrix}
-1 & 1 & 0\\
1 & 1 & 0\\
0 & 0 & -1\\
1 & -1 & -1\\
-1 & -1 & 1\\
1 & 0 & -1
\end{pmatrix}
\end{equation}

Y la función de activación
\begin{equation}
tanh = \frac{e^{2x} - 1}{e^{2x} + 1}
\end{equation}

Usando estos paramátros escriba los pasos para calcular la representación (vector) resultante de aplicar la operación de convolución (sin padding) + max pooling. ¿De qué tamaño sería la ventana que debemos usar?

**Respuesta**

Para aplicar la operación de convolución, es necesario definir una serie de ventanas de tamaño $k$ que contengan los vectores de embedding de cada palabra, multiplicada por los filtros, al no tener padding, la primera ventana comenzará con la palabra `El`.

Como cada palabra su vector de embedding es de dimensión $2$. Y la matriz de filtro tiene una dimensión de $6\times 3$, por tanto, para que se pueda aplicar la matriz correctamente en cada ventana, se deberá tener ventanas de tamaño $3$, otorgándole un tamaño final de $6$ al concatenar los vectores de embedding.

\begin{equation}
V_1 = {\text{El, agua, moja}} = \begin{pmatrix}
2 & 2 & 0 & -2 & 0 & 1
\end{pmatrix}\\
V_2 = {\text{agua, moja, y}} = \begin{pmatrix}
0 & -2 & 0 & 1 & -2 & 1
\end{pmatrix}\\
V_3 = {\text{moja, y, el}} = \begin{pmatrix}
0 & 1 & -2 & 1 & 1 & 0
\end{pmatrix}\\
V_4 = {\text{y, el, fuego}} = \begin{pmatrix}
-2 & 1 & 1 & 0 & -1 & 1
\end{pmatrix}\\
V_5 = {\text{el, fuego, quema}} = \begin{pmatrix}
1 & 0 & -1 & 1 & 1 & 1
\end{pmatrix}\\
\end{equation}

Cada una de estas ventanas, es multiplicada con el filtro y luego aplicada la función de activación `tanh`

$V_1 \times U = \begin{pmatrix} -1 & 6 & 1 \end{pmatrix} \to tanh(V_1\times U) = \begin{pmatrix} tanh(-1) & tanh(6) & tanh(1) \end{pmatrix} = \begin{pmatrix} -0.76158416 & 0.99998771 & 0.76159416 \end{pmatrix}$

$tanh(V_2\times U) = \begin{pmatrix} 0.96402758 & -0.76159416 & -0.9993293 \end{pmatrix}$

$tanh(V_3\times U) = \begin{pmatrix} 0.76159416 & -0.76159416 & 0.96402758 \end{pmatrix}$

$tanh(V_4\times U) = \begin{pmatrix} 0.9999092 & 0. & -0.99505475 \end{pmatrix}$

$tanh(V_5\times U) = \begin{pmatrix} 0. & -0.76159416 & 0. \end{pmatrix}$

Finalmente, se aplica el max pooling, que consiste en tomar el valor máximo de cada dimensión formando un nuevo vector 3D, el resultado entregado:

\begin{pmatrix} 0.9999092 & 0.99998771 & 0.96402758 \end{pmatrix}

In [ ]:
import numpy as np

U = np.array([[-1,1,0],[1,1,0],[0,0,-1],[1,-1,-1],[-1,-1,1],[1,0,-1]])

v1 = np.array([[2,2,0,-2,0,1]])
v2 = np.array([[0,-2,0,1,-2,1]])
v3 = np.array([[0,1,-2,1,1,0]])
v4 = np.array([[-2,1,1,0,-1,1]])
v5 = np.array([[1,0,-1,1,1,1]])



r1 = np.matmul(v1,U)
r2 = np.matmul(v2,U)
r3 = np.matmul(v3,U)
r4 = np.matmul(v4,U)
r5 = np.matmul(v5,U)

print(np.tanh(r1))
print(np.tanh(r2))
print(np.tanh(r3))
print(np.tanh(r4))
print(np.tanh(r5))



[[-0.76159416  0.99998771  0.76159416]]
[[ 0.96402758 -0.76159416 -0.9993293 ]]
[[ 0.76159416 -0.76159416  0.96402758]]
[[ 0.9999092   0.         -0.99505475]]
[[ 0.         -0.76159416  0.        ]]


# Recurrent Neural Networks (1 punto)


### Pregunta 4 (0,5 puntos)
Usando los embeddings de dos dimensiones de la pregunta anteror, la oración `el fuego quema` la podemos representar por una secuencia de vectores $(\vec{x}_1,\vec{x}_2,\vec{x}_3)$, con $\vec{x}_i \in \mathbb{R}^{d_x}$ y $d_x=2$.

Tenemos una red recurrente *Elman* definidad como:
\begin{equation}
\begin{split}
\vec{s}_i &= R_{SRNN}\left (\vec{x}_i, \vec{s}_{i-1}\right ) = g \left (\vec{s}_{i-1}W^s + \vec{x}_i W^x + \vec{b}\right ) \\
\vec{y}_i &= O_{SRNN}\left(\vec{s}_i\right) = \vec{s}_i \\
\end{split}
\end{equation}
donde
\begin{equation}
\vec{s}_i, \vec{y}_i \in \mathbb{R}^{d_s}, \quad W^x \in \mathbb{R}^{d_x \times d_s}, \quad W^s \in \mathbb{R}^{d_s \times d_s}, \quad \vec{b} \in \mathbb{R}^{d_s},
\end{equation}
y los vectores de estado $s_i$ son de tres dimensiones, $ds= 3$.

Sea
\begin{equation}
\begin{split}
\vec{s}_0 &= [0,0,0]\\
W^x &= \begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} \\
W^s &= \begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} \\
\vec{b} &= [0, 0, 0] \\
g(x) &= ReLu(x) = max(0, x)
\end{split}
\end{equation}

<br>

Calcule manualmente los valores de los vectores $\vec{s}_1, \vec{s}_2,\vec{s}_3$ y de $\vec{y}_1, \vec{y}_2,\vec{y}_3$.

**Respuesta**

Se calculan los vectores secuencialmente:

$\vec{s}_1 = R_{SRNN}(\vec{x}_1,\vec{s}_0) = g(\vec{s}_0W^s + \vec{x}_1W^x+\vec{b}) = g([0,0,0]\begin{pmatrix}1 & 0 & 1\\ 0 & 1 & -1\\ 1 & 1 & 1\end{pmatrix} + [1, 0] \begin{pmatrix}0 & 0 & 1\\ 1 & -1 & 0\end{pmatrix} + [0,0,0]) = g([0,0,0] + [0,0,1]) = g([0,0,1]) = [0,0,1]$

$\vec{s}_2 = R_{SRNN}(\vec{x}_2,\vec{s}_1) = g(\vec{s}_1W^s + \vec{x}_2W^x+\vec{b}) = g([0,0,1]\begin{pmatrix}1 & 0 & 1\\ 0 & 1 & -1\\ 1 & 1 & 1\end{pmatrix} + [-1, 1] \begin{pmatrix}0 & 0 & 1\\ 1 & -1 & 0\end{pmatrix} + [0,0,0]) = g([1,1,1] + [1,-1,-1]) = g([2,0,0]) = [2,0,0]$

$\vec{s}_3 = R_{SRNN}(\vec{x}_3,\vec{s}_2) = g(\vec{s}_2W^s + \vec{x}_3W^x+\vec{b}) = g([2,0,0]\begin{pmatrix}1 & 0 & 1\\ 0 & 1 & -1\\ 1 & 1 & 1\end{pmatrix} + [1, 1] \begin{pmatrix}0 & 0 & 1\\ 1 & -1 & 0\end{pmatrix} + [0,0,0]) = g([2,0,2] + [1,-1,1]) = g([3,-1,3]) = [3,0,3]$


Luego, simplemente $\vec{y}_i = \vec{s}_i, \forall i \in \{1,2,3\}$, entonces los resultados finales son:

$\vec{y}_1 = \vec{s}_1 = [0,0,1]$

$\vec{y}_2 = \vec{s}_2 = [2,0,0]$

$\vec{y}_3 = \vec{s}_3 = [3,0,3]$

### Pregunta 5 (0.5 puntos)
¿De qué forma las RNN y las CNN logran aprender representaciones específicas
para la tarea objetivo? Compare la forma en que las RNN y las CNN aprenden con los modelos que usan *features* diseñadas manualmente.

**Respuesta**

Las RNN y las CNN aprenden representaciones específicas para la tarea objetivo a través de su capacidad para capturar dependencias a largo plazo en secuencias (RNN) o extraer características relevantes localmente en datos estructurados (CNN). En contraste con los modelos que utilizan características diseñadas manualmente, las RNN y las CNN aprenden automáticamente representaciones directamente de los datos, lo que les permite adaptarse mejor a la complejidad y variabilidad de los datos y obtener un mejor rendimiento en tareas desafiantes.

# Pregunta 6: Redes Neuronales con Pytorch (3 puntos) 💬

<center>
<img src="https://www.anda.cl/wp-content/uploads/2021/03/0_5vNAtimPjYQr4W72.gif" alt="chatbot" width="400">
</center>

En esta sección de la tarea deberán implementar un Chatbot que sea capaz de generar una conversación *“básica”* utilizando un dataset de *Star Wars*. **El objetivo** de esta pregunta es que puedan aplicar lo aprendido sobre redes neuronales utilizando Pytorch en un ejemplo práctico.  Durante el desarrollo, se espera que puedan diseñar un bot (que tendrá por atrás un clasificador) que sea capaz de clasificar diferentes etiquetas, cosa que una vez identificada la etiqueta entregue una respuesta acorde a lo preguntado.

**Aviso:** Antes de comenzar con una descripción mas profunda de esta sección, les recomendamos que visualicen y se familiaricen con el dataset entregado, de esta forma comprenderán mejor la descripción del enunciado (aquí una pequeña ayudita 🆘).

In [ ]:
import pandas as pd

example_data = pd.read_json('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json')
print("Cantidad de tags: ", example_data['intents'].shape[0])

Cantidad de tags:  16


A continuación, ejemplos del contenido del primer registro:

In [ ]:
example_data['intents'][0]['patterns']

['Hi',
 'Hey',
 'How are you',
 'Is anyone there?',
 'Hello',
 'Good day',
 "What's up",
 'Yo!',
 'Howdy',
 'Nice to meet you.']

In [ ]:
example_data['intents'][0]['responses']

['Hey',
 'Hello, thanks for visiting.',
 'Hi there, what can I do for you?',
 'Hi there, how can I help?',
 'Hello, there.',
 'Hello Dear',
 'Ooooo Hello, looking for someone or something?',
 'Yes, I am here.',
 'Listening carefully.',
 'Ok, I am with you.']

In [ ]:
example_data['intents'][0]['tag']

'greeting'

Del dataset cargado podemos notar que este viene en un formato `JSON`, por lo que sus datos están almacenados en diccionarios. Las llaves de los diccionarios no son aleatorias y estos nos sirven para identificar puntos relevantes en el desarrollo del bot. A continuación, se realiza una pequeña descripción de las llaves:

- `patterns`: Almacena los patrones con los que entrenaremos el modelo 😮, en otras palabras, es el corpus de entrenamiento que contiene solo preguntas o expresiones que deberá responder el bot.
- `responses`: Son las respuestas 🙋 relacionadas a los `patterns`, estas las utilizaremos en una etapa posterior a la clasificación, para dar una respuesta aleatoría al usuario.
- `tag`: Son las labels con las que entrenaremos nuestro modelo 💻.

En síntesis, las `keys` relevantes para el entrenamiento de nuestra red neuronal serán `patterns` (corpus) y `tag` (etiquetas).

#### Explicación de la tarea a realizar:

**Explicación de la tarea a realizar**: Implemente una Class llamada `CNNClassifier` que sea capaz de entrenar un modelo de texto a través de una red neuronal Feed Forward y una arquitectura convolucional (CNN 1D) [`torch.nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#conv1d) . Para el diseño de las redes tienen completa libertad, pero se le aconseja que se guíen de la última auxiliar para la construcción. Es **importantísimo** que el modelo a crear posea una capa de `Embedding` que se genere en base al entrenamiento del modelo. Creado el modelo, construya una función batch para cargar los datos de entrenamiento del modelo.

Construido el modelo, compare los resultados obtenidos para una red feed forward y una cnn. Para la comprobación de sus resultados ejecute el chatbot y pruebelo, ¿qué configuración tiene mejores resultados?, ¿a qué se deberan estos resultados?

Ojo que un ejemplo de prueba con el chatbot puede ser (agregue mas preguntas ud):

```
Let's chat! (type 'finish_chat' to finish the chat)
You: hi
GA-97: Yes, I am here.
You: can you tell me a joke?
GA-97: Have you tried the gluten-free Wookiee treats? No, but I heard they are a little Chewy.
```

El resto del código referido a la ejecución del chat se los entregamos, por lo que no deberían tener mayores problemas 😸 (en caso de tener problemas con su código, puede modificar cualquier parte sugerida siempre y cuando cumpla lo solicitado).

**Igual [mucho texto](https://i0.wp.com/elgeneracionalpost.com/wp-content/uploads/2020/07/mucho-texto.jpg?fit=1280%2C720&ssl=1).... En resumen, ¿Qué se solicita?:**

- [ ] Diseñar una red neuronal Feed Forward.
- [ ] Diseñar un red convolucional.
- [ ] Utilizar una capa de embeddings para generar representaciones vectoriales del corpus.
- [ ] Crear el método forward de la clase `CNNClassifier`.
- [ ] Crear la función BATCH.
- [ ] Probar el modelo y comparar los resultados obtenidos con la red Feed Forward y la red CNN. Comente sus resultados de forma cualitativa, señalando con qué tipo de red obtuvo mejores resultados con el chatbot.

**Nota-1:** El modelo creado debe tener la opción de entrenar a traves de una feed forward y una CNN. Esto no significa que entrenará una FF y una CN, el modelo deberá recibir un booleano que especifique que tipo de red utilizará.

**Nota-2:** El dataset se descargará automáticamente en la sección `Carga de Dataset 📚`, no os preocupéis.

#### Pasemos al Código 🦾

Esqueleto propuesto (se **RECOMIENDA** que cambien **SOLO** la red neuronal y la función Batch) 🦴:

##### Instalamos librerias necesarias e importamos 😀

In [ ]:
# Esto toma su tiempo en ejecutarse
%%capture
!pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.9.0

In [ ]:
import os
import sys
import json
import torch
import random
from random import choice
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torch.optim import SGD, lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

from itertools import zip_longest

import plotly.express as px

import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer

##### Carga de Dataset 📚

In [ ]:
# we obtain the dataset
!wget 'https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json'

--2023-06-18 23:10:23--  https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14469 (14K) [text/plain]
Saving to: ‘star_wars_chatbot.json’

star_wars_chatbot.j 100%[===================>]  14.13K  --.-KB/s    in 0s      

2023-06-18 23:10:23 (39.6 MB/s) - ‘star_wars_chatbot.json’ saved [14469/14469]



In [ ]:
# Load the dataset using json
with open('star_wars_chatbot.json', 'r') as f:
    dataset = json.load(f)

# Create a vocab with the dataset and get the number of classes that have
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(tokenizer(x) for list_words in dataset['intents'] for x in list_words['patterns'])
num_classes = len(dataset['intents'])

# Define a list with the labels
labels = sorted(set([tag for tag in [intents['tag'] for intents in dataset['intents']]]))
# Define a train_list where we can find the info in the format: [(tag_0, text_0)...,(tag_n-1, text_n-1)]
train_list = [(labels.index(intents['tag']), text) for intents in dataset['intents'] for text in intents['patterns']]

In [ ]:
from itertools import zip_longest

labels = list({doc[0] for doc in train_list})
label_map = {label: index for index, label in enumerate(labels)}
#creamos lista de tensores
train_dataset = [
        (
            item[0], #numero de la clase
            torch.tensor([vocab[token] for token in tokenizer(item[1])]),
        ) for item in train_list
    ]

#train_dataset = list(zip(labels,
#                    torch.tensor([item[1] for item in train_list])))


In [ ]:
print(train_list[0:5])

[(7, 'Hi'), (7, 'Hey'), (7, 'How are you'), (7, 'Is anyone there?'), (7, 'Hello')]


In [ ]:
print(train_dataset[0:5])

[(7, tensor([93])), (7, tensor([92])), (7, tensor([95, 20,  3])), (7, tensor([ 12,  71, 121,   2])), (7, tensor([90]))]


In [ ]:
print(labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [ ]:
human_text = ''
for i in range(20):
  human_text += vocab.get_itos()[i] + ' '
print(f"text: {human_text}")

text: . ? you me i help in ! can a who is need tell am do what for mission are 


##### Creación del modelo (2 puntos en total)

In [ ]:
UNK_IDX = 0
vocab.set_default_index(UNK_IDX)

vocab.insert_token('<pad>', 1)

num_class = len(labels) # Se puede borrar

In [ ]:
# Construya el modelo
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, num_classes=10,
                 use_cnn=False, cnn_pool_channels=24, cnn_kernel_size=1):
      super().__init__()
      # Creamos la capa de embedding
      self.use_cnn = use_cnn
      if self.use_cnn:
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Creamos la capa de convolución
        # `in_channels`: Es el número de canales de entrada de la convolución. En este caso, como estamos trabajando con texto, sólo tenemos un canal, por lo que `in_channels=1`.
        # `out_channels`: Es el número de canales de salida de la convolución. Especifica la cantidad de filtros que se aplicarán a la entrada. En este caso, queremos generar `cnn_pool_channels` canales de salida, por lo que `out_channels=cnn_pool_channels`.
        # `kernel_size`: Es el tamaño del kernel de la convolución. En este caso, estamos usando un kernel de tamaño `cnn_kernel_size * embed_dim`, donde `embed_dim` es la dimensión de los vectores de embedding. Esto significa que cada filtro de la convolución cubrirá `cnn_kernel_size` palabras (o tokens) en una dimensión y `embed_dim` en la otra.
        # `stride`: Es el desplazamiento que se aplica a la entrada de la convolución. En este caso, estamos desplazando la entrada `embed_dim` unidades en cada paso. Esto significa que se aplicarán filtros a cada palabra (o token) de la entrada.
        self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=cnn_pool_channels,
            kernel_size=cnn_kernel_size * embed_dim,
            stride=embed_dim,
        )

        # Calculamos el tamaño de entrada de la capa lineal
        fc_in_size = cnn_pool_channels

        # Creamos la capa lineal
        self.fc = nn.Linear(fc_in_size, num_classes)

        # Inicializamos los pesos de las capas
        self.init_weights()
      else:
        self.embedding = nn.Embedding(vocab_size, embed_dim, 1) # 1 el índice de pad

        # capas de la MLP
        self.fc = nn.Linear(embed_dim, num_classes)

    def init_weights(self):
      # Definimos el rango de los valores iniciales de los pesos
      initrange = 0.5

      # Inicializamos los pesos de la capa de embedding
      self.embedding.weight.data.uniform_(-initrange, initrange)

      # Inicializamos los pesos de la capa lineal
      self.fc.weight.data.uniform_(-initrange, initrange)

      # Inicializamos los sesgos de la capa lineal en cero
      self.fc.bias.data.zero_()

    def forward(self, text, offsets):

      # Preparamos el input de la capa de embeddings a partir de text y offsets
      text = torch.tensor(
          list(
              zip(
                  *zip_longest(
                      *([text[o:offsets[i+1]] for i, o in enumerate(offsets[:-1])] + [text[offsets[-1]:len(texts)]]),
                      fillvalue=vocab["<pad>"]
                  )
              )
          )
      ).to(text.device)

      # Obtenemos la representación de la frase a partir de la capa de embedding
      h = self.embedding(text)

      # Aplicamos la capa de convolución
      h = h.view(h.size(0), 1, -1)
      h = torch.relu(self.conv(h))
      h = h.mean(dim=2)

      # Obtenemos el resultado final a partir de la capa lineal
      output = self.fc(h)

      # Aplicamos la función de activación log-softmax
      return nn.functional.log_softmax(output, dim=1)


In [ ]:
class ArgumentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, pad_idx):
        super().__init__()

        # capa de embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim, pad_idx) # 1 x 100
        # self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, mode="mean")

        # capas de la MLP
        self.fc = nn.Linear(embed_dim, num_class) # 100 x 16
        # self.fc1 = nn.Linear(embed_dim, hidden_size)
        # self.fc2 = nn.Linear(hidden_size, num_class)


    def forward(self, batch):
        # La representacion de un documento sera el promedio de los
        # embeddings de sus palabras.
        # (B, N, 1) -> (B, N, E)
        # print(f"batch: {batch.shape}")
        h = self.embedding(batch)
        # print(f"h1: {h.shape}")
        # (B, N, E) -> (B, E)
        h = h.mean(dim=1)
        # print(f"h2: {h.shape}")
        # h = self.embedding(batch)

        # computar las capas de la red MLP
        h = self.fc(h)
        # print(f"h3 {h.shape}")
        # h = F.relu(self.fc1(h))
        # h = self.fc2(h)

        return h
        # return torch.softmax(h, -1)

##### Función Batch 👷 (0,5 puntos)

In [ ]:
# Defina su función de BATCH
# output:  (label, texts)
def generate_batch_forward(batch):
  return (
        # En este caso como los labels son números,
        # el tensor es de una sola dimension de tamaño batch_size
        torch.tensor([item[0] for item in batch]),

        # En este caso se retorna un tensor de 2 dimensiones, batch_size x N,
        # donde N es mayor largo de los ejemplo en el batch. Aca se realiza
        # padding de los ejemplos mas cortos.
        torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *[item[1] for item in batch], fillvalue=vocab["<pad>"]
                    )
                )
            )
        ),
    )

# output: (texts, offsets, label)
def generate_batch_CNN(batch):
  stoi = vocab.get_stoi()
  label = torch.tensor([entry[0] for entry in batch]) # Acá es distinto del aux
  texts = [tokenizer(entry[1]) for entry in batch]
  offsets = [0] + [len(text) for text in texts]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  big_text = torch.cat([torch.tensor([vocab[t] if t in stoi else 0 for t in text]) for text in texts])
  #big_text = torch.cat([torch.tensor([vocab.stoi[t] for t in text]) for text in texts])

  return big_text, offsets, label


In [ ]:
batch = generate_batch_forward(train_dataset[1:10])
batch[1]

tensor([[ 92,   1,   1,   1,   1],
        [ 95,  20,   3,   1,   1],
        [ 12,  71, 121,   2,   1],
        [ 90,   1,   1,   1,   1],
        [ 55,  52,   1,   1,   1],
        [ 17,  38,  44, 126,   1],
        [130,   8,   1,   1,   1],
        [ 96,   1,   1,   1,   1],
        [ 60,  65, 102,   3,   0]])

In [ ]:
train_dataset[1:10]

[(7, tensor([92])),
 (7, tensor([95, 20,  3])),
 (7, tensor([ 12,  71, 121,   2])),
 (7, tensor([90])),
 (7, tensor([55, 52])),
 (7, tensor([ 17,  38,  44, 126])),
 (7, tensor([130,   8])),
 (7, tensor([96])),
 (7, tensor([ 60,  65, 102,   3,   0]))]

##### Entrenamiento 🥊

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU is avaible: {device}")

# Define the different inputs in our model
num_epochs = 1000
BATCH_SIZE = 16
LR = 1e-1
INPUT_SIZE = len(vocab)
OUTPUT_SIZE = num_classes
USE_CNN = True

# Define model, optimizer, loss and scheduler (Q: ¿What is it?)
model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=True).to(device)
optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: .9 ** (epoch // 10)])

print(f'train: {len(train_list)} elements')

# We train the model using the intents
loss_list= []
for epoch in range(1, num_epochs):
  train_loader = DataLoader(train_list, batch_size=BATCH_SIZE, collate_fn=generate_batch_CNN)
  model.train()
  total_loss = 0
  for i, (texts, offsets, cls) in enumerate(train_loader):
    texts = texts.to(device)
    offsets = offsets.to(device)
    cls = cls.to(device)
    optimizer.zero_grad()
    #output = model(text) # Forward
    output = model(texts, offsets) # CNN
    #print(cls)
    #print(cls+1)
    loss = criterion(output, cls)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  loss_list.append(loss.item())
  sys.stdout.write('\rEpoch: {0:03d} \t iter-Loss: {1:.3f}'.format(epoch+1, loss.item()))

print(f'final loss: {loss.item():.4f}')

##### Feed Forward

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU is avaible: {device}")

# Define the different inputs in our model
num_epochs = 1000
BATCH_SIZE = 16
LR = 1e-1
INPUT_SIZE = len(vocab)
OUTPUT_SIZE = num_classes
USE_CNN = False

# Define model, optimizer, loss and scheduler (Q: ¿What is it?)
model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=False).to(device)

# model = ArgumentClassifier(
#     vocab_size=len(vocab),
#     embed_dim=100,
#     num_class=num_classes,
#     hidden_size=1024,
#     pad_idx=vocab["<pad>"],
# ).to(device)

optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: .9 ** (epoch // 10)])

print(f'train: {len(train_list)} elements')

# We train the model using the intents
loss_list= []
for epoch in range(1, num_epochs):
  train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch_forward)
  model.train()
  total_loss = 0
  enum = enumerate(train_loader)
  #print(list(train_loader))
  #continue

  #for i, (cls, offsets, texts) in enumerate(train_loader):
  for i, (cls, texts) in enumerate(train_loader):
    texts = texts.to(device)
    #offsets = offsets.to(device)
    cls = cls.to(device)
    optimizer.zero_grad()
    output = model(texts) # Forward
    loss = criterion(output, cls)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  loss_list.append(loss.item())
  sys.stdout.write('\rEpoch: {0:03d} \t iter-Loss: {1:.3f}'.format(epoch+1, loss.item()))

print(f'final loss: {loss.item():.4f}')

GPU is avaible: cpu
train: 97 elements


TypeError: ignored

In [ ]:
# Ahora creamos funciones para entrenar y validar el modelo
from torch.utils.data import DataLoader


def train_func(train_dataset):

    # Entranamos el modelo
    train_loss = 0
    train_acc = 0
    data = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=generate_batch_forward,
    )
    for i, (cls, text) in enumerate(data):
        optimizer.zero_grad()
        cls, text = cls.to(device), text.to(device)
        output = model(text)

        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        train_acc += (output.argmax(1) == cls).sum().item()

    # Ajustar el learning rate
    # scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)


def test(test_dataset):
    test_loss = 0
    acc = 0
    data = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch_forward
    )
    for cls, text in data:
        cls, text = cls.to(device), text.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            test_loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(test_dataset), acc / len(test_dataset)

In [ ]:
# Ahora por fin tenemos todo lo necesario para entrenar el modelo.
import time

N_EPOCHS = 20
LEARN_RATE = 1e-1
STEP_SIZE = 1
BATCH_SIZE = 16
EMBED_DIM = 99
HIDDEN_SIZE = 1024

device = "cuda" if torch.cuda.is_available() else "cpu"


# Define the different inputs in our model
num_epochs = 2
BATCH_SIZE = 16
LR = 1e-1
INPUT_SIZE = len(vocab)
OUTPUT_SIZE = num_classes

# Define model, optimizer, loss and scheduler (Q: ¿What is it?)
model = ArgumentClassifier(
    vocab_size=len(vocab),
    embed_dim=EMBED_DIM,
    num_class=num_classes,
    hidden_size=HIDDEN_SIZE,
    pad_idx=vocab["<pad>"],
).to(device)

optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset[3:])
    #valid_loss, valid_acc = test(validation_dataset)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    # print(
    #     f"\tLoss: {valid_loss:.4f}(valid)\t|"
    #     f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    # )

batch: torch.Size([16])
h1: torch.Size([16, 99])
h2: torch.Size([16])


RuntimeError: ignored

In [ ]:
train_dataset[3]

(7, tensor([95, 20,  3]))

In [ ]:
train_list[0]

(7, 'Hi')

In [ ]:
train_dataset[0]

(7, tensor([93]))

##### A probar! 🧪

In [ ]:
# This is working?, Try the next example!
qText = "'Do you know any joke?'" # this must classify the label "funny"

X = torch.tensor([vocab.get_stoi()[t] for t in tokenizer(qText)]).to(device)

model.eval()
output = model(X, torch.tensor([0], dtype=torch.long).to(device))
_, predicted = torch.max(output, dim=1)
labels[predicted]

'funny'

In [ ]:
print(labels)

['Menu', 'about me', 'alive', 'bounti hounter', 'creator', 'funny', 'goodbye', 'greeting', 'help', 'jedi', 'mission', 'myself', 'sith', 'stories', 'tasks', 'thanks']


Ya pero prometiste hacer un chatbot, no una simple clasificación....

##### Guardamos modelo 🦺 (opcional)

In [ ]:
# We save de model using pytorch (this is optional, just to learn how to do this in pytorch)
data = {
"model_state": model.state_dict(),
"input_size": INPUT_SIZE,
"output_size": OUTPUT_SIZE,
"use_cnn": USE_CNN,
"labels": labels
        }

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


##### Chatbot 💬

In [ ]:
hdevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('star_wars_chatbot.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

INPUT_SIZE = data["input_size"]
OUTPUT_SIZE = data["output_size"]
USE_CNN = data["use_cnn"]
labels = data['labels']
model_state = data["model_state"]

model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=USE_CNN).to(device)
model.load_state_dict(model_state)
model.eval()

# Dictionary with the answers
responses = {key['tag']: key['responses'] for key in dataset['intents']}

bot_name = "GA-97"
print("Let's chat! (type 'finish_chat' to finish the chat)")
while True:
    q_text = input("You: ")
    q_text = q_text
    if q_text == 'finish_chat':
        break

    X = torch.tensor([vocab.get_stoi()[t] for t in tokenizer(q_text)]).to(device)

    output = model(X, torch.tensor([0], dtype=torch.long).to(device))
    _, predicted = torch.max(output, dim=1)

    tag = labels[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.50:
      print(f"tag: {tag}")
      print(f"{bot_name}: {random.choice(responses[tag])}")
    else:
      print(f"{bot_name}: My model can't understand you...")

Let's chat! (type 'finish_chat' to finish the chat)
You: hi
tag: greeting
GA-97: Hello, there.
You: tell me a joke
tag: bounti hounter
GA-97: Jango Fett, Boba Fett, Cad Bane, Durge, Embo, Dengar, Black Krrsantan, IG-88, Aurra Sing, Sabine Wren.
You: tell
tag: bounti hounter
GA-97: I will advise you to look for Jango Fett, Boba Fett, Cad Bane, Durge, Embo, Dengar, Black Krrsantan, IG-88, Aurra Sing, Sabine Wren.
You: joke
tag: funny
GA-97: What’s Yoda’s advice for going to the bathroom? Doo-doo or doo-doo-not-do.
You: story
tag: stories
GA-97: You would get bored if I do so.
You: bye
tag: goodbye
GA-97: Well, hope see you soon!
You: hello
tag: greeting
GA-97: Hello Dear
You: menu please
tag: Menu
GA-97: Menu: Fuzzy Tauntaun, Bloody Rancor, Jedi Mind Trick, T-16 Skyhopper, Yub Nub, Jet Juice, Hyperdrive, Rancor Beer.
You: sdsdsfdsf


KeyError: ignored

#### Comente los resultados aquí (0,5 puntos)

``Comente los resultados aquí``